# Amazon Personalize ハンズオン
* 本ハンズオンは Personalize で レコメンデーションするために必要な工程を一通り体験するハンズオンです
* 実行環境の前提として SageMaker Notebook Instance で Jupyter Notebook を使用します
* 構成は下記の通り

## 手順概要

0. SageMaker Notebook Instance を立ち上げ、Jupyter Notebook を開いてこのノートブックを開く
1. Personalize にデータを取り込むための S3 バケットを作成する
2. S3 のバケットポリシーを設定し、 Amazon Personalize のサービスが アップロードしたデータにアクセスできるようにする
3. Amazon Personalize の実行ロールを作成し、Amazon Personalize のサービスが使えるようにする
4. S3 にデータをアップロードする
5. S3 から Personalize へデータをインポートするためにスキーマ定義を行い、インポートする
6. ソリューションとバージョンを作成する(学習)
7. 作成したソリューションバージョンからキャンペーンを作成する（ホスティング）
8. 作成したキャンペーンでレコメンデーション機能を試す

<img src="./media/diagram.png">

## 環境準備

### 本ハンズオンで利用するライブラリの読み込み

In [ ]:
!pip install jsonlines
import boto3, json, time, jsonlines
from datetime import date,datetime
import pandas as pd
import numpy as np
print(f'Current boto3 Version ={boto3.__version__}')

上記セルを実行して、boto3 が 1.20.21 未満の場合、以下のセルのコメントアウトを解除してから実行してください。実行が完了したら、上にあるメニューから [Kernel] -> [Restart] を選択してカーネルを再起動してください。

In [ ]:
# !pip install -U "boto3==1.20.21"

### 学習データ保存用S3バケットの作成
#### 概要
* Personalizeがデータインポート時、推論結果出力時に利用するS3バケットの作成と設定を行う
* 作成したバケットに対して、Amazon Personlize のサービスがアクセスできるようバケットポリシーを設定する

* 本ハンズオンでは personlize-handson-${yourname}-YYYY-MM-DD-hh-mm-ss というバケット名を作成して、利用する
* 下記セルの yourname 変数に任意の名前を入力する

#### 設定値を変数に格納する

In [ ]:
# 各設定値を変数に格納する

#------下記変数をご自身の名前で必ず設定ください------#
#----------------------------------------------------#
yourname = 'gokazu' #   <- ここに記入をお願いします
#----------------------------------------------------#
#----------------------------------------------------#

# region は 東京リージョンを利用します
region = boto3.session.Session().region_name

# 説明分の通りのバケット名を bucket_name 変数に格納します
timestamp = datetime.now().strftime('-%Y-%m-%d-%H-%M-%S')
bucket_name = 'personalize-handson-' + yourname + timestamp
print(f'本ハンズオンで利用する\nバケットは "{bucket_name}"')
print(f'リージョンは "{region}"\nです')

#### bucket 作成
boto3 の s3 [client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.BucketPolicy.put) の [create_bucket](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket) メソッド を利用して作成する

In [ ]:
s3_client = boto3.client('s3')
location = {'LocationConstraint': region}

if region == 'us-east-1':
    response = s3_client.create_bucket(
        Bucket = bucket_name
    )
else:
    response = s3_client.create_bucket(
        Bucket = bucket_name,
        CreateBucketConfiguration = location
    )
print(json.dumps(response, indent=2))

#### バケットポリシーの設定
* Amazon Personalize のサービスが作成した S3 バケットにアクセスできるようバケットポリシーを設定する
* バケットポリシー は [BucketPolicy](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.ServiceResource.BucketPolicy) の [put](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.BucketPolicy.put) メソッドで json 形式で設定する

In [ ]:
# bucket policy 設定
bucket_policy = boto3.resource('s3').BucketPolicy(bucket_name)

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3Bucket AccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::" + bucket_name,
                "arn:aws:s3:::" + bucket_name +"/*"
            ]
        }
    ]
}

policy = json.dumps(policy)

response = bucket_policy.put(
    Policy=policy
)
print(json.dumps(response, indent=2))

#### key_prefix の設定
本ハンズオンで利用する bucket の key_prefix を設定する

In [ ]:
# 後続処理で利用するS3のprefixを指定
s3_key_prefix = 'personalize-demo' + timestamp + '/'
print(f'今日のハンズオンで利用する key prefix は "{s3_key_prefix}" です')

### Personalize実行ロールの作成
* [Amazon Personalize 用の IAM ロールの作成](https://docs.aws.amazon.com/ja_jp/personalize/latest/dg/aws-personalize-set-up-permissions.html#set-up-required-permissions)パートの手順に従ってPersonalizeの実行ロールを作成する
* IAM ロールの作成は、 boto3 の [create_role](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#IAM.Client.create_role) メソッドを利用して作成する
* 作成した IAM ロールに AmazonPersonalizeFullAccess ポリシーを [attach_role_policy](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#IAM.Client.attach_role_policy) でアタッチする
* 作成した Personalize ロールの ARN を変数に格納する

In [ ]:
# ロール作成
iam = boto3.client('iam')

assume_role_policy_document = {
    'Version': '2012-10-17',
    'Statement': [{'Sid': '','Effect': 'Allow','Principal': {'Service': 'personalize.amazonaws.com'},'Action': 'sts:AssumeRole'}]
}

response = iam.create_role(
    RoleName = 'personalizeRole' + timestamp,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document),
    Description='using Amazon Personalize handson '+ timestamp.replace('-',''),
    MaxSessionDuration=3600*12 # 12時間
)
role_arn = response['Role']['Arn']
role_name = response['Role']['RoleName']

print(f'作成した role の\narn は "{role_arn}"')
print(f'名前は "{role_name}"\nです')

In [ ]:
# Policy attach
response = iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess'
)
print(json.dumps(response, indent=2))

### Personalize boto3 Clientの設定

In [ ]:
# Personalize を操作するための client を生成
personalize = boto3.client(service_name='personalize')
personalize_runtime = boto3.client(service_name='personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')

## データの確認

#### サンプルデータ
* Movielensの10万件の評価データを Personalize で利用できるように事前にデータを加工済
* どのようなデータ形式になっているかを確認する
* 処理内容については本ハンズオンでは扱わないが、[こちら](./data_preparation.ipynb)のノートブックに記載

##### Interaction Data
* ユーザがいつどの映画を見たのかを示すデータ
* レコメンドに評価の低かった映画は出したくないので、レートが4以上のものにフィルタしている
* 必須項目は USER_ID,ITEM_ID,TIMESTAMP

In [ ]:
pd.read_csv('./data/interaction.csv').sort_values(['USER_ID','TIMESTAMP']).reset_index().drop(columns='index').head(50)

##### Item Data
* アイテムの属性を表すデータ
* 複数の属性を含む場合は一つの列に|(パイプ)で区切って格納する
* 必須項目は ITEM_ID と、1 つ以上の属性情報(今回は GENRE )

In [ ]:
pd.read_csv('./data/item.csv').head(50)

##### User Data
* ユーザの属性を表すデータ
* 必須項目は USER_ID と、1 つ以上の属性情報
* このデータでは年齢、性別、職業、郵便番号を含む

In [ ]:
pd.read_csv('./data/user.csv').head(50)

##### Interaction Data, Item Data, User Data の 3 つを S3 にアップロードする
* 確認した 3 ファイルを事前に設定したバケット、key_prefix にアップロードする
* アップロードには [upload_file](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Object.upload_file) を用いる(アップロードするためのメソッドは多種あるがアップロードされればどれでもよい）

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(s3_key_prefix + 'interaction.csv').upload_file('./data/interaction.csv')
boto3.Session().resource('s3').Bucket(bucket_name).Object(s3_key_prefix + 'item.csv').upload_file('./data/item.csv')
boto3.Session().resource('s3').Bucket(bucket_name).Object(s3_key_prefix + 'user.csv').upload_file('./data/user.csv')

### Personalize 環境の準備

まずはデータをインポートするためにデータのスキーマ定義とデータセットグループの作成を行う

#### インタラクションスキーマの定義
* インタラクションデータの csv に沿ったデータスキーマを設定
* 必須項目はアップロードした csv 同様、USER_ID, ITEM_ID, TIME_STAMP の 3 種

インタラクションスキーマの[参考情報](https://docs.aws.amazon.com/personalize/latest/dg/schema-examples-interactions.html)

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")

interaction_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "EVENT_VALUE",
            "type": [
             "float",
             "null"
          ]
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_interaction_schema_response = personalize.create_schema(
    name = "DEMO-movielens-interaction-schema" + time_str,
    schema = json.dumps(interaction_schema)
)

interaction_schema_arn = create_interaction_schema_response['schemaArn']
print(json.dumps(create_interaction_schema_response, indent=2))

#### アイテムスキーマの定義

* アイテムデータの csv に沿ったデータスキーマを設定
* 必須項目はアップロードした csv 同様、ITEM_ID の他、1 つ以上のメタデータ情報

アイテムスキーマの[参考情報](https://docs.aws.amazon.com/personalize/latest/dg/schema-examples-items.html)

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")

item_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "GENRE",
            "type": "string",
            "categorical": True
        }
    ],
    "version": "1.0"
}

create_item_schema_response = personalize.create_schema(
    name = "DEMO-movielens-item-schema" + time_str,
    schema = json.dumps(item_schema)
)

item_schema_arn = create_item_schema_response['schemaArn']
print(json.dumps(create_item_schema_response, indent=2))

#### ユーザースキーマの定義

* ユーザデータの csv に沿ったデータスキーマを設定
* 必須項目はアップロードした csv 同様、USER_ID の他、1 つ以上のメタデータ情報

ユーザスキーマの[参考情報](https://docs.aws.amazon.com/personalize/latest/dg/schema-examples-users.html)

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")

user_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "AGE",
            "type": "int"
        },
        {
            "name": "GENDER",
            "type": "string",
            "categorical": True
        },
        {
            "name": "JOB",
            "type": "string",
            "categorical": True
        },
        {
            "name": "ZIP",
            "type": "string",
            "categorical": True
        }
    ],
    "version": "1.0"
}

create__user_schema_response = personalize.create_schema(
    name = "DEMO-movielens-user-schema" + time_str,
    schema = json.dumps(user_schema)
)

user_schema_arn = create__user_schema_response['schemaArn']
print(json.dumps(create__user_schema_response, indent=2))

##### データグループの作成
* インタラクションデータ、アイテムデータ、ユーザデータを束ねるデータグループを作成
* [参考情報](https://docs.aws.amazon.com/personalize/latest/dg/data-prep-ds-group.html)
* [create_dataset_group](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.create_dataset_group) メソッドはデータグループを作成するためのジョブを実行する非同期メソッド
* 完了を確認するために、別途 [describe_dataset_group](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.describe_dataset_group) を定期的に呼び出してジョブの実行結果を確認しにいく

ただし、 データグループ作成自体はすぐに完了（秒レベル）する

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-movielens-dataset-group" + timestamp
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

In [ ]:
%%time
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print(f"CreateDatasetGroup: {status}")
        break
    else:
        print('.',end='')
        time.sleep(1)

#### インタラクションデータセット の作成
データセットグループの中にインタラクションデータセットを作成する  
必要な情報は下記 4 つ
* データセットのタイプ（この場合はインタラクションデータであることを指定）
* 親となるデータセットグループの arn
* 事前に定義したインタラクションデータのスキーマの arn
* 作成するデータセットのユニークな名前

詳細 : [create_dataset](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.create_dataset)

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")

interaction_dataset_type = "Interactions"
create_interaction_dataset_response = personalize.create_dataset(
    datasetType = interaction_dataset_type,
    datasetGroupArn = dataset_group_arn,
    name = 'DEMO-movielens-interactions',
    schemaArn = interaction_schema_arn
)

interaction_dataset_arn = create_interaction_dataset_response['datasetArn']
print(json.dumps(create_interaction_dataset_response, indent=2))

#### インタラクションデータセットのインポート
作成したインタラクションデータセットにインタラクションデータをインポートする(ただし事前に dataset が作成し終えていないとエラーで落ちるため、dataset の作成が完了するまで待つ処理を入れている)  
必須項目は下記 4 つ
* インポートジョブ名
* インタラクションデータセットの arn
* インポートするデータの S3 URI
* インポートジョブを実行するロールの arn

[create_dataset_import_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.create_dataset_import_job) はジョブを実行するだけの非同期メソッドのため、完了確認は別途行う

In [ ]:
while True:
    status = personalize.describe_dataset(datasetArn=interaction_dataset_arn)['dataset']['status']
    if status == 'ACTIVE':
        print('!')
        print(status)
        break
    else:
        print('.',end='')
        time.sleep(5)

now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")

create_interaction_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-interaction-dataset-import-job" + time_str,
    datasetArn = interaction_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, s3_key_prefix + 'interaction.csv')
    },
    roleArn = role_arn
)

interaction_dataset_import_job_arn = create_interaction_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interaction_dataset_import_job_response, indent=2))

#### ユーザデータセットの作成

データセットグループの中にユーザデータセットを作成する  
必要な情報は下記 4 つ
* データセットのタイプ（この場合はユーザデータであることを指定）
* 親となるデータセットグループの arn
* 事前に定義したユーザデータのスキーマの arn
* 作成するデータセットのユニークな名前

詳細 : [create_dataset](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.create_dataset)

In [ ]:
user_dataset_type = "Users"
create_user_dataset_response = personalize.create_dataset(
    datasetType = user_dataset_type,
    datasetGroupArn = dataset_group_arn,
    name = 'DEMO-movielens-user',
    schemaArn = user_schema_arn
)

user_dataset_arn = create_user_dataset_response['datasetArn']
print(json.dumps(create_user_dataset_response, indent=2))

#### ユーザーデータセットのインポート

作成したユーザデータセットにインタラクションデータをインポートする(ただし事前に dataset が作成し終えていないとエラーで落ちるため、dataset の作成が完了するまで待つ処理を入れている)  
必須項目は下記 4 つ
* インポートジョブ名
* ユーザデータセットの arn
* インポートするデータの S3 URI
* インポートジョブを実行するロールの arn

[create_dataset_import_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.create_dataset_import_job) はジョブを実行するだけの非同期メソッドのため、完了確認は別途行う

In [ ]:
while True:
    status = personalize.describe_dataset(datasetArn=user_dataset_arn)['dataset']['status']
    if status == 'ACTIVE':
        print('!')
        print(status)
        break
    else:
        print('.',end='')
        time.sleep(5)

now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")

create_user_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-user-dataset-import-job" + time_str,
    datasetArn = user_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, s3_key_prefix + 'user.csv')
    },
    roleArn = role_arn
)

user_dataset_import_job_arn = create_user_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_user_dataset_import_job_response, indent=2))

#### アイテムデータセットの作成

データセットグループの中にアイテムデータセットを作成する
必要な情報は下記 4 つ
* データセットのタイプ（この場合はアイテムデータであることを指定）
* 親となるデータセットグループの arn
* 事前に定義したアイテムデータのスキーマの arn
* 作成するデータセットのユニークな名前

詳細 : [create_dataset](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.create_dataset)

In [ ]:
item_dataset_type = "Items"
create_item_dataset_response = personalize.create_dataset(
    datasetType = item_dataset_type,
    datasetGroupArn = dataset_group_arn,
    name = 'DEMO-movielens-item',
    schemaArn = item_schema_arn
)

item_dataset_arn = create_item_dataset_response['datasetArn']
print(json.dumps(create_item_dataset_response, indent=2))

#### アイテムデータセットのインポート

作成したアイテムデータセットにインタラクションデータをインポートする(ただし事前に dataset が作成し終えていないとエラーで落ちるため、dataset の作成が完了するまで待つ処理を入れている)  
必須項目は下記 4 つ
* インポートジョブ名
* アイテムデータセットの arn
* インポートするデータの S3 URI
* インポートジョブを実行するロールの arn

[create_dataset_import_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.create_dataset_import_job) はジョブを実行するだけの非同期メソッドのため、完了確認は別途行う

In [ ]:
while True:
    status = personalize.describe_dataset(datasetArn=item_dataset_arn)['dataset']['status']
    if status == 'ACTIVE':
        print('!')
        print(status)
        break
    else:
        print('.',end='')
        time.sleep(5)

now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")

create_item_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-item-dataset-import-job" + time_str,
    datasetArn = item_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, s3_key_prefix + 'item.csv')
    },
    roleArn = role_arn
)

item_dataset_import_job_arn = create_item_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_item_dataset_import_job_response, indent=2))

#### インタラクションデータのインポート完了を待つ
* create_dataset_import_job はインポートジョブをキックするだけの非同期メソッド
* 終わるまでソリューションのバージョンの作成が出来ないため、終了を確認する処理を行う
* describe_dataset_import_job でジョブのステータスが確認できるので、 ACTIVE になるまで待つ
* [describe_dataset_import_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.describe_dataset_import_job) に必要な引数は データセットインポートジョブの arn のみ
* 下記セルはインタラクションデータセットのインポートジョブの完了確認だが、アイテムもユーザも同様
* 15分程度待つ

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interaction_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    status = dataset_import_job["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print(f"LatestDatasetImportJobRun: {status}")
        break
    else:
        print('.',end='')
        
    time.sleep(60)

#### ユーザーデータのインポート完了を待つ

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = user_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    status = dataset_import_job["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print(f"LatestDatasetImportJobRun: {status}")
        break
    else:
        print('.',end='')
        
    time.sleep(5)

#### アイテムデータのインポート完了を待つ

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = item_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    status = dataset_import_job["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print(f"LatestDatasetImportJobRun: {status}")
        break
    else:
        print('.',end='')
        
    time.sleep(5)

### ソリューションの作成
* 各ソリューションのトレーニング完了までには一定時間掛かるため、レシピ毎にトレーニングジョブを並列で起動し、後続のステップですべてのトレーニングジョブが完了するのを待つ
* ソリューションを作成するにあたって、各ソリューションに使うレシピの arn が必要なので最初にレシピの arn を取得する

In [ ]:
import boto3
paginator = personalize.get_paginator('list_recipes')
recipe_arns = {}
for page in paginator.paginate():
    for recipe in page['recipes']:
        recipe_arns[recipe['name']] = recipe['recipeArn']
print(recipe_arns)

#### ソリューションの作成（user-personalization）
* user-personalization レシピを使ったソリューションを [create_solution](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.create_solution)メソッドを使って作成する
* create_solution メソッドの必須項目は下記の通り
  * ソリューション のユニークな名前
  * データセットグループの arn
  * AutoMLを利用しないときはレシピの arn
* performHPO は現状デフォルトで False だが、念の為明示的に False を指定
  * HPO は時間がかかるため、本ハンズオンでは使用しない
* この段階ではあくまでソリューションの箱を作るだけで実際に学習を開始するのは次の処理

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")

user_personalization_create_solution_response = personalize.create_solution(
    name = "DEMO-movielens-user-personalization" + time_str,
    datasetGroupArn = dataset_group_arn,
    recipeArn = 'arn:aws:personalize:::recipe/aws-user-personalization',
    performHPO = False
)

user_personalization_solution_arn = user_personalization_create_solution_response['solutionArn']
print(json.dumps(user_personalization_create_solution_response, indent=2))

#### user-personalization ソリューションのバージョンの作成
* **時間のかかる処理をキックします**
* 非同期メソッドの実行なのでバージョン作成の完了確認は別途実行
* [create_solution_verion](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.create_solution_version) の実行には 作成したソリューション の arn を指定するだけでよい

In [ ]:
user_personalization_create_solution_version_response = personalize.create_solution_version(
    solutionArn=user_personalization_solution_arn
)

In [ ]:
user_personalization_solution_version_arn = user_personalization_create_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_create_solution_version_response, indent=2))

#### personalized-ranking ソリューション/ソリューションバージョンの作成

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")
# ソリューション作成
personalized_ranking_create_solution_response = personalize.create_solution(
    name = "DEMO-movielens-personalized-ranking" + time_str,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arns["aws-personalized-ranking"],
    performHPO = False
)
personalized_ranking_solution_arn = personalized_ranking_create_solution_response['solutionArn']
print(json.dumps(personalized_ranking_create_solution_response, indent=2))
# ソリューションバージョン作成
personalized_ranking_create_solution_version_response = personalize.create_solution_version(solutionArn=personalized_ranking_solution_arn)
personalized_ranking_solution_version_arn = personalized_ranking_create_solution_version_response['solutionVersionArn']
print(json.dumps(personalized_ranking_create_solution_version_response, indent=2))

#### sims ソリューション/ソリューションバージョンの作成

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")
# ソリューション作成
sims_create_solution_response = personalize.create_solution(
    name = "DEMO-movielens-sims" + time_str,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arns["aws-sims"],
    performHPO = False
)
sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
# ソリューションバージョン作成
sims_create_solution_version_response = personalize.create_solution_version(solutionArn=sims_solution_arn)
sims_solution_version_arn = sims_create_solution_version_response['solutionVersionArn']
print(json.dumps(sims_create_solution_version_response, indent=2))

#### affinity ソリューション/ソリューションバージョンの作成

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")
# ソリューション作成
affinity_create_solution_response = personalize.create_solution(
    name = "DEMO-movielens-affinity" + time_str,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arns["aws-item-affinity"],
    performHPO = False
)
affinity_solution_arn = affinity_create_solution_response['solutionArn']
print(json.dumps(affinity_create_solution_response, indent=2))
# ソリューションバージョン作成
affinity_create_solution_version_response = personalize.create_solution_version(solutionArn=affinity_solution_arn)
affinity_solution_version_arn = affinity_create_solution_version_response['solutionVersionArn']
print(json.dumps(affinity_create_solution_version_response, indent=2))

#### attribute affinity ソリューション/ソリューションバージョンの作成

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")

attribute_affinity_create_solution_response = personalize.create_solution(
    name = "DEMO-movielens-attribute-affinity" + time_str,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arns["aws-item-attribute-affinity"],
    performHPO = False
)
# ソリューション作成
attribute_affinity_solution_arn = attribute_affinity_create_solution_response['solutionArn']
print(json.dumps(attribute_affinity_solution_arn, indent=2))
# ソリューションバージョン作成
attribute_affinity_create_solution_version_response = personalize.create_solution_version(solutionArn=attribute_affinity_solution_arn)
attribute_affinity_solution_version_arn = attribute_affinity_create_solution_version_response['solutionVersionArn']
print(json.dumps(attribute_affinity_create_solution_version_response, indent=2))

#### user-personalization レシピのトレーニング完了を待つ
* ソリューションのバージョン作成は非同期メソッドのため完了を確認する処理を別途行う
* [describe_solution_version](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.describe_solution_version) でステータスを取得可能
* 引数は ソリューションバージョンの arn のみ

**40 分くらい待ちます**

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = user_personalization_solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print(f"user-personalize solution version : {status}")
        break
    else:
        print('.',end='')
        
    time.sleep(60)

#### personalized-ranking レシピのトレーニング完了を待つ

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = personalized_ranking_solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print("personalize-ranking solution version : {}".format(status))
        break
    else:
        print('.',end='')
        
    time.sleep(5)

#### sims レシピのトレーニング完了を待つ

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = sims_solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print("sims solution version : {}".format(status))
        break
    else:
        print('.',end='')
        
    time.sleep(5)

#### affinity レシピのトレーニング完了を待つ

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = affinity_solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print("sims solution version : {}".format(status))
        break
    else:
        print('.',end='')
        
    time.sleep(5)

#### attribute affinity レシピのトレーニング完了を待つ

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = attribute_affinity_solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print("sims solution version : {}".format(status))
        break
    else:
        print('.',end='')
        
    time.sleep(5)

#### user-personalization ソリューションメトリックの取得
* 学習したソリューションのバージョンのメトリックを [get_solution_metrics](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.get_solution_metrics) で取得することが可能
* 引数は ソリューションバージョンの arn のみ
* metrics についての概要は [URL](https://docs.aws.amazon.com/ja_jp/personalize/latest/dg/working-with-training-metrics.html) を参考

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

#### personalized-ranking ソリューションメトリックの取得

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = personalized_ranking_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

#### sims ソリューションメトリックの取得

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

#### affinity ソリューションメトリックの取得

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = affinity_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

#### attribute affinity ソリューションメトリックの取得

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = attribute_affinity_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

### キャンペーンの作成
* 各キャンペーン作成完了までには一定時間掛かるため、キャンペーンの作成をを並列実行し、後続のステップですべてのキャンペーン作成完了を待つ
* キャンペーンを作成することでレコメンデーションができるようになる

#### user-personalization キャンペーンの作成
* キャンペーンの作成は [create_campaign](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.create_campaign) メソッドを使う(非同期メソッドのため完了は別途確認)
* 必須項目は 3 つ
  * キャンペーンのユニークな名前
  * 適用するソリューションバージョンの arn
  * 最低限確保する TPS ( 1 秒あたりにさばけるトランザクション量）
  * [exploration](https://docs.aws.amazon.com/ja_jp/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html) の設定について

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")

create_user_personalization_campaign_response = personalize.create_campaign(
    name = "DEMO-campaign-user-personalization" + time_str,
    solutionVersionArn = user_personalization_solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig =  { 
      "itemExplorationConfig": { 
         "explorationWeight" : "0.3",
         "explorationItemAgeCutOff" : "30.0",
      }
   },
)

user_personalization_campaign_arn = create_user_personalization_campaign_response['campaignArn']
print(json.dumps(create_user_personalization_campaign_response, indent=2))

#### sims キャンペーンの作成

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")

create_sims_campaign_response = personalize.create_campaign(
    name = "DEMO-campaign-sims" + time_str,
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = create_sims_campaign_response['campaignArn']
print(json.dumps(create_sims_campaign_response, indent=2))

#### personalized-ranking キャンペーンの作成

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")

create_personalized_ranking_campaign_response = personalize.create_campaign(
    name = "DEMO-campaign-personalized-ranking" + time_str,
    solutionVersionArn = personalized_ranking_solution_version_arn,
    minProvisionedTPS = 1
)

personalized_ranking_campaign_arn = create_personalized_ranking_campaign_response['campaignArn']
print(json.dumps(create_personalized_ranking_campaign_response, indent=2))

#### user-personalization キャンペーンの作成完了待ち
* キャンペーン作成のステータスは [describe_campaign](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.describe_campaign) で取得できる
* キャンペーンの arn を指定するのみ
* キャンペーンの作成が完了するまで 10 分程度かかる

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = user_personalization_campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print(f"LatestDatasetImportJobRun: {status}")
        break
    else:
        print('.',end='')
        
    time.sleep(60)

#### sims キャンペーンの作成完了待ち

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = sims_campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print("LatestDatasetImportJobRun: {}".format(status))
        break
    else:
        print('.',end='')
        
    time.sleep(5)

#### personalizad-ranking キャンペーンの作成完了待ち

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = personalized_ranking_campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print("LatestDatasetImportJobRun: {}".format(status))
        break
    else:
        print('.',end='')
        
    time.sleep(5)

### APIの呼び出し
* ここからは、実際にレコメンデーションをする
* レコメンデーションは boto3 の personalize-runtime を用いる

In [ ]:
# 事前にアイテムとユーザのデータを読み込む
item_data = pd.read_csv('./ml-100k/u.item', sep='|', encoding='latin-1', header=None)
item_data = item_data.drop(columns=3)
item_data = item_data.rename(columns={0:'ITEM_ID', 1:'TITLE', 2:'RELEASE', 4:'IMDB_URL', 5:'unknown', 6:'Action', 7:'Adventure', 8:'Animation', 9:"Children's", 10:'Comedy', 11:'Crime', 12:'Documentary', 13:'Drama', 14:'Fantasy', 15:'Film-Noir', 16:'Horror', 17:'Musical', 18:'Mystery', 19:'Romance', 20:'Sci-Fi', 21:'Thriller', 22:'War', 23:'Western'})
item_data.head()

In [ ]:
user_data = pd.read_csv('./ml-100k/u.user', sep='|', names=['USER_ID', 'AGE', 'GENDER', 'JOB', 'ZIP'])
user_data.head(11)

#### ユーザー向けレコメンデーションAPIの呼び出し
* ユーザ ID が 10 の人には何をレコメンデーションするべきか、というサンプル
* [get_recommendations](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize-runtime.html#PersonalizeRuntime.Client.get_recommendations) メソッドに、 USER-PERSONALIZATION レシピを使用しているキャンペーンの arn と、 ユーザ ID を指定する
* レコメンデーションを返してくれるアイテム数はデフォルトで 25 だが、最大で 500 まで増やせる

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = user_personalization_campaign_arn,
    userId = '10'
)

item_list = get_recommendations_response['itemList']
title_list = [item_data.loc[item_data['ITEM_ID'] == np.int(item['itemId'])].values[0][1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))

#### 類似アイテムAPIの呼び出し

* [get_recommendations](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize-runtime.html#PersonalizeRuntime.Client.get_recommendations) メソッドを sims のキャンペーンの arn を指定し、類似アイテムの元となる itemId を指定して呼び出す

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = '30' # ItemId が 30 の商品の類似商品を出す
)

item_list = get_recommendations_response['itemList']
title_list = [item_data.loc[item_data['ITEM_ID'] == np.int(item['itemId'])].values[0][1] for item in item_list]

print(item_data.loc[item_data['ITEM_ID'] == 30].TITLE.values[0])
print("Recommendations: {}".format(json.dumps(title_list, indent=2)))

#### Personalized Ranking APIの呼び出し
* 勧めたい商品が複数あったときにどの優先順位で勧めればよいかを知る
* [get_personalized_ranking](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize-runtime.html#PersonalizeRuntime.Client.get_personalized_ranking) を利用する
* キャンペーンの arn, ユーザ ID, 優先順位を知りたいアイテムのリストを引数に呼び出す

In [ ]:
input_item_list = [str(i) for i in range(1,11)]

get_personalized_ranking_response = personalize_runtime.get_personalized_ranking(
    campaignArn = personalized_ranking_campaign_arn,
    userId = '20', # UserId が 20 の人にとってのランキング
    inputList = input_item_list
)

item_list = get_personalized_ranking_response['personalizedRanking']
item_id_list = [item['itemId'] for item in item_list]

print("PersonalizedRanking: {}".format(json.dumps(item_id_list, indent=2)))

### リアルタイムイベント処理
#### EventTrackerの作成
* リアルタイムのお客様のアクションを取り込んでレコメンデーションを行うための、EventTracker を作成する
* [create_event_tracker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.create_event_tracker) メソッドで作成できる
* データセットグループの arn と、ユニークな名前を指定して実行する
* 30 秒程度で完了する

In [ ]:
create_event_tracker_response = personalize.create_event_tracker(
    datasetGroupArn = dataset_group_arn,
    name = 'DEMO-event-tracker'
)

print(json.dumps(create_event_tracker_response, indent=2))

In [ ]:
event_tracker_arn = create_event_tracker_response['eventTrackerArn']

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn = event_tracker_arn
    )
    status = describe_event_tracker_response["eventTracker"]["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print(f'status:{status}')
        break
    print('.',end = '')
    time.sleep(5)
tracking_id = create_event_tracker_response['trackingId']
print(f'tracking_id: {tracking_id}')

#### レコメンデーションのリストを取得
* 比較のため、まずは Event Tracker を使わずにレコメンデーションする

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = user_personalization_campaign_arn,
    userId = '10'
)

item_list = get_recommendations_response['itemList']
title_list = [item_data.loc[item_data['ITEM_ID'] == np.int(item['itemId'])].values[0][1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))

#### リアルタイムイベントを Feed
* ユーザのリアルタイムイベントは [put_event](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize-events.html#PersonalizeEvents.Client.put_events) メソッドで Feed することができる
* 今回は user_id が 10 の人が item_id が 215 の映画について 5 というレートをつけたイベントを feed する

In [ ]:
# USER_ID が 10 のユーザ
user_data[user_data['USER_ID']==10]

In [ ]:
# ITEM_ID が 215 の映画
item_data[item_data['ITEM_ID']==215]

In [ ]:
import uuid
from datetime import datetime

now = datetime.now()

sentAt = str(now.timestamp())
int(now.timestamp())

session_id = str(uuid.uuid4())
event_id = str(uuid.uuid4())

In [ ]:
tracking_id = create_event_tracker_response['trackingId']
print(tracking_id)

In [ ]:
personalize_events.put_events(
    trackingId = tracking_id,
    userId = '10',
    sessionId = session_id,
    eventList = [
      {
          "eventId": event_id,
          "sentAt": int(now.timestamp()),
          "eventType": "RATING",
          "properties": json.dumps(
            {
                'itemId': '215',
                'eventValue': 5
            })
      }
    ]
)

#### レコメンデーションのリストを再取得
* put_event すると 結果が一部変わる（ことがある）

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = user_personalization_campaign_arn,
    userId = '10'
)

item_list = get_recommendations_response['itemList']
title_list = [item_data.loc[item_data['ITEM_ID'] == np.int(item['itemId'])].values[0][1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))

### 推論フィルターの利用
#### 推論フィルターの作成
ここではジャンルでの絞り込みができる推論フィルターを作成
* [create_filter](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.create_filter) メソッドを使う
    * filterExpression 引数にクエリを記載する
    * クエリの書き方は [URL](https://docs.aws.amazon.com/personalize/latest/dg/filter-expressions.html) を参照
* フィルタ作成も非同期メソッドのため、確認は [describe_filter](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize.html#Personalize.Client.describe_filter) を利用する
    * 作成に 1 ~ 2 分かかる
* filter の expression に変数を用いることが可能になった
    * フィルタを作成する際に ($GENRE) のように推論時にいろいろな値を入れられるようにする変数を事前に宣言し、推論する際に filterValues 引数に変数に格納する値を指定する

In [ ]:
filter_expression = 'INCLUDE ItemID WHERE Items.GENRE IN ($GENRE)'

create_filter_response = personalize.create_filter(
    datasetGroupArn = dataset_group_arn,
    filterExpression = filter_expression,
    name = 'genre_filter_action'
)
filter_arn = create_filter_response['filterArn']
print(json.dumps(create_filter_response, indent=2))

In [ ]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_filter_response = personalize.describe_filter(
        filterArn = filter_arn
    )
    status = describe_filter_response["filter"]["status"]
    print('.',end='')
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print(f"Filter: {status}")
        break
        
    time.sleep(5)

#### 推論フィルターの有無で結果を比較
* [get_recommendations](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/personalize-runtime.html#PersonalizeRuntime.Client.get_recommendations) メソッドを呼び出す際に、filterArn 引数に 作成したフィルタの arn を指定する

In [ ]:
# 推論フィルタ無し
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = user_personalization_campaign_arn,
    userId = '10'
)

item_list = get_recommendations_response['itemList']
title_list = [item_data.loc[item_data['ITEM_ID'] == np.int(item['itemId'])].values[0][1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))

In [ ]:
item_data[item_data['TITLE'].isin(title_list)]

In [ ]:
# 推論フィルタあり ( Action のみ)
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = user_personalization_campaign_arn,
    userId = '10',
    filterArn = filter_arn,
    filterValues={ "GENRE": "\"Action\""} # Action のみに絞る
)

item_list = get_recommendations_response['itemList']
title_list = [item_data.loc[item_data['ITEM_ID'] == np.int(item['itemId'])].values[0][1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))

In [ ]:
item_data[item_data['TITLE'].isin(title_list)]

---
## ここから先はハンズオン完了後余裕のある方向け
### バッチ推論

#### 推論用入力データの作成

In [ ]:
user_list = []
user_list.append({"userId": "3"})
user_list.append({"userId": "10"})
user_list.append({"userId": "15"})
print(user_list)

with jsonlines.open('batch_infrence_input.json', mode='w') as writer:
    writer.write_all(user_list)

!head batch_infrence_input.json
boto3.Session().resource('s3').Bucket(bucket_name).Object(s3_key_prefix + 'batch-inference-input/batch_infrence_input.json').upload_file('batch_infrence_input.json')

#### バッチ推論ジョブの実行

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")
create_batch_inference_job_response = personalize.create_batch_inference_job (
    solutionVersionArn = user_personalization_solution_version_arn,
    jobName = "userpersonalization-batch-inference-job" + time_str,
    roleArn = role_arn,
    jobInput = 
       {"s3DataSource": {"path": "s3://" + bucket_name + "/" + s3_key_prefix + "batch-inference-input/batch_infrence_input.json"}},
    jobOutput = 
       {"s3DataDestination": {"path": "s3://" + bucket_name + "/" + s3_key_prefix + "batch-inference-output/"}},
    numResults = 100
)
batchInferenceJobArn = create_batch_inference_job_response['batchInferenceJobArn']
describe_batch_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )

describe_batch_inference_job_response

In [ ]:
%%time

# 待つ
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_batch_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_batch_inference_job_response["batchInferenceJob"]["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print(f"status: {status}")
        break
    else:
        print('.',end='')
        time.sleep(60)

#### バッチ推論結果の取得・表示

In [ ]:
output_path = describe_batch_inference_job_response['batchInferenceJob']['jobOutput']['s3DataDestination']['path'] + 'batch_infrence_input.json.out'
!aws s3 cp {output_path} .
!head batch_infrence_input.json.out

#### affniate バッチセグメントジョブの実行

In [ ]:
item_list = []
for i in range(1,101):
    item_list.append({"itemId": str(i)})

with jsonlines.open('affinity_input.json', mode='w') as writer:
    writer.write_all(item_list)

!head affinity_input.json
boto3.Session().resource('s3').Bucket(bucket_name).Object(s3_key_prefix + 'affinity-input/affinity_input.json').upload_file('affinity_input.json')


In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")
create_batch_segment_job_response = personalize.create_batch_segment_job(
    jobName = "affinity-batch-segment-job" + time_str,
    solutionVersionArn = affinity_solution_version_arn,
    numResults=100,
    jobInput={'s3DataSource': {"path": "s3://" + bucket_name + "/" + s3_key_prefix + "affinity-input/affinity_input.json"}},
    jobOutput = {"s3DataDestination": {"path": "s3://" + bucket_name + "/" + s3_key_prefix + "affinity-output/"}},
    roleArn=role_arn
)
batchSegmentJobArn = create_batch_segment_job_response['batchSegmentJobArn']
describe_batch_segment_job_response = personalize.describe_batch_segment_job(
        batchSegmentJobArn = batchSegmentJobArn
    )

describe_batch_segment_job_response

In [ ]:
%%time

# 待つ
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_batch_segment_job_response = personalize.describe_batch_segment_job(
        batchSegmentJobArn = batchSegmentJobArn
    )
    status = describe_batch_segment_job_response["batchSegmentJob"]["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print(f"status: {status}")
        break
    else:
        print('.',end='')
        time.sleep(60)

In [ ]:
output_path = describe_batch_segment_job_response['batchSegmentJob']['jobOutput']['s3DataDestination']['path'] + 'affinity_input.json.out'
!aws s3 cp {output_path} .
!head affinity_input.json.out

#### attribute affniate バッチセグメントジョブの実行
* input の形式は [Developer Guide](https://docs.aws.amazon.com/personalize/latest/dg/batch-data-upload.html)に例の記載あり。Schema 情報に合わせる。
* 条件式も記載できる(AND/ORなど)

In [ ]:
genre_list_work = []
for genres in pd.read_csv('./data/item.csv')['GENRE'].unique().tolist():
    temp = genres.split('|')
    for t in temp:
        genre_list_work.append(t)
genre_list_work = list(set(genre_list_work))[:10]
print(genre_list_work)

In [ ]:
genre_list = []
for genre in genre_list_work:
    genre_list.append({"itemAttributes": "Items.GENRE = \""+genre + "\""})
    

with jsonlines.open('attribute_affinity_input.json', mode='w') as writer:
    writer.write_all(genre_list)

!head attribute_affinity_input.json
boto3.Session().resource('s3').Bucket(bucket_name).Object(s3_key_prefix + 'attribute-affinity-input/attribute_affinity_input.json').upload_file('attribute_affinity_input.json')

In [ ]:
now = datetime.now()
time_str = now.strftime("-%Y-%m-%d-%H-%M-%S")
create_batch_segment_job_response = personalize.create_batch_segment_job(
    jobName = "attribute-affinity-batch-segment-job" + time_str,
    solutionVersionArn = attribute_affinity_solution_version_arn,
    numResults=10,
    jobInput={'s3DataSource': {"path": "s3://" + bucket_name + "/" + s3_key_prefix + "attribute-affinity-input/attribute_affinity_input.json"}},
    jobOutput = {"s3DataDestination": {"path": "s3://" + bucket_name + "/" + s3_key_prefix + "attribute-affinity-output/"}},
    roleArn=role_arn
)
batchSegmentJobArn = create_batch_segment_job_response['batchSegmentJobArn']
describe_batch_segment_job_response = personalize.describe_batch_segment_job(
        batchSegmentJobArn = batchSegmentJobArn
    )

describe_batch_segment_job_response

In [ ]:
%%time

# 待つ
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_batch_segment_job_response = personalize.describe_batch_segment_job(
        batchSegmentJobArn = batchSegmentJobArn
    )
    status = describe_batch_segment_job_response["batchSegmentJob"]["status"]
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        print('!')
        print(f"status: {status}")
        break
    else:
        print('.',end='')
        time.sleep(60)

In [ ]:
output_path = describe_batch_segment_job_response['batchSegmentJob']['jobOutput']['s3DataDestination']['path'] + 'attribute_affinity_input.json.out'
!aws s3 cp {output_path} .
!head attribute_affinity_input.json.out

In [ ]:
# personalize.describe_schema(schemaArn=item_schema_arn)

## お片付け

In [ ]:
personalize.delete_filter(filterArn = filter_arn)
while True:
    try:
        status = personalize.describe_filter(filterArn=filter_arn)['filter']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break

In [ ]:
personalize.delete_event_tracker(eventTrackerArn=event_tracker_arn)
while True:
    try:
        status = personalize.describe_event_tracker(eventTrackerArn=event_tracker_arn)['eventTracker']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break

In [ ]:
personalize.delete_campaign(campaignArn = user_personalization_campaign_arn)
personalize.delete_campaign(campaignArn = personalized_ranking_campaign_arn)
personalize.delete_campaign(campaignArn = sims_campaign_arn)

In [ ]:
while True:
    try:
        status = personalize.describe_campaign(campaignArn = user_personalization_campaign_arn)['campaign']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break
while True:
    try:
        status = personalize.describe_campaign(campaignArn = personalized_ranking_campaign_arn)['campaign']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break
while True:
    try:
        status = personalize.describe_campaign(campaignArn = sims_campaign_arn)['campaign']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break

In [ ]:
personalize.delete_solution(solutionArn=user_personalization_solution_arn)
personalize.delete_solution(solutionArn=personalized_ranking_solution_arn)
personalize.delete_solution(solutionArn=sims_solution_arn)
personalize.delete_solution(solutionArn=affinity_solution_arn)
personalize.delete_solution(solutionArn=attribute_affinity_solution_arn)

In [ ]:
while True:
    try:
        status = personalize.describe_solution(solutionArn=user_personalization_solution_arn)['solution']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break
while True:
    try:
        status = personalize.describe_solution(solutionArn=personalized_ranking_solution_arn)['solution']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break
while True:
    try:
        status = personalize.describe_solution(solutionArn=sims_solution_arn)['solution']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break    
while True:
    try:
        status = personalize.describe_solution(solutionArn=affinity_solution_arn)['solution']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break    
while True:
    try:
        status = personalize.describe_solution(solutionArn=attribute_affinity_solution_arn)['solution']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break    

In [ ]:
personalize.delete_dataset(datasetArn=interaction_dataset_arn)
personalize.delete_dataset(datasetArn=user_dataset_arn)
personalize.delete_dataset(datasetArn=item_dataset_arn)

In [ ]:
while True:
    try:
        status = personalize.describe_dataset(datasetArn=interaction_dataset_arn)['dataset']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break
while True:
    try:
        status = personalize.describe_dataset(datasetArn=user_dataset_arn)['dataset']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break
while True:
    try:
        status = personalize.describe_dataset(datasetArn=item_dataset_arn)['dataset']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break

In [ ]:
personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)
while True:
    try:
        status = personalize.describe_dataset_group(datasetGroupArn=dataset_group_arn)['datasetGroup']['status']
        print('.',end='')
        time.sleep(1)
    except:
        print('!')
        print('deleted')
        break

In [ ]:
personalize.delete_schema(schemaArn=interaction_schema_arn)
personalize.delete_schema(schemaArn=user_schema_arn)
personalize.delete_schema(schemaArn=item_schema_arn)

In [ ]:
iam.detach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess'
)
iam.delete_role(
    RoleName = role_name
)

In [ ]:
!aws s3 rb s3://{bucket_name} --force